<a href="https://colab.research.google.com/github/Varun9213/CE-903-DEC/blob/main/Demo_DEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple Clustering==0.0.10

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
  Found existing installation: Clustering 0.0.9
    Uninstalling Clustering-0.0.9:
      Successfully uninstalled Clustering-0.0.9


In [4]:
from Clustering.Clustering import DEC

In [5]:
# import numpy as np
# import pandas as pd

# import tensorflow as tf
# import tensorflow.keras.backend as K
# import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import SGD, Adam
# from tensorflow.keras.models import Model


from sklearn.datasets import load_iris, load_digits
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd

In [6]:
fea, tar = load_digits(return_X_y=True)
fea = np.array(fea)
tar = np.array(tar)

In [7]:
%mkdir iris_DEC

In [8]:
dec = DEC([fea.shape[1],128,64,64,32,32],nclusters=10)
dec.autoencoder.summary()

Model: "Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          [(None, 64)]              0         
_________________________________________________________________
encoding_layer_0 (Dense)     (None, 128)               8320      
_________________________________________________________________
encoding_layer_1 (Dense)     (None, 64)                8256      
_________________________________________________________________
encoding_layer_2 (Dense)     (None, 64)                4160      
_________________________________________________________________
encoding_layer_3 (Dense)     (None, 32)                2080      
_________________________________________________________________
encoder_output (Dense)       (None, 32)                1056      
_________________________________________________________________
decoding_layer_4 (Dense)     (None, 32)                

In [9]:
dec.train_autoencoder(fea,epochs=5000,dir="/content/iris_DEC",optimizer=Adam(learning_rate=0.001))

loss : 0.65339
Weights saved to /content/iris_DEC/autoencoder_weights.h5


In [10]:
dec.compile(optimizer=Adam(learning_rate=0.001), loss='kld')
dec.fit(fea, tar, epochs=1000, dir="/content/iris_DEC")

Initializing cluster centers with k-means.........
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ (InputLayer)          [(None, 64)]              0         
_________________________________________________________________
encoding_layer_0 (Dense)     (None, 128)               8320      
_________________________________________________________________
encoding_layer_1 (Dense)     (None, 64)                8256      
_________________________________________________________________
encoding_layer_2 (Dense)     (None, 64)                4160      
_________________________________________________________________
encoding_layer_3 (Dense)     (None, 32)                2080      
_________________________________________________________________
encoder_output (Dense)       (None, 32)                1056      
_________________________________________________________________
cluster_la

In [11]:
y_pred = dec.predict(fea)

In [13]:
y_pred

array([7, 8, 5, ..., 8, 6, 6])

In [14]:
cm = metrics.cluster.contingency_matrix(tar,y_pred)
cm

array([[  2,   0,   0,   0,   0,   0,   0, 176,   0,   0],
       [  0,  56,   0,   0,   0,  27,   1,   0,  98,   0],
       [  0,   1,   0,   0,   0, 174,   1,   0,   1,   0],
       [  0,   0, 160,   0,   4,   1,   6,   0,   9,   3],
       [175,   0,   0,   0,   0,   0,   0,   0,   0,   6],
       [  1,   0,   4,   1, 176,   0,   0,   0,   0,   0],
       [  0,   0,   0, 179,   0,   0,   0,   1,   1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   2, 177],
       [  0,   2,   0,   0,   2,   0,  92,   0,  78,   0],
       [  0,  21, 137,   0,   2,   0,   8,   0,   3,   9]])

In [15]:
def acc_clustering(cm,y):
  df = pd.DataFrame(cm)
  true = 0
  max_val = []
  for col in df:
    sorted = df[col].sort_values(ascending = False)
    i = 0
    while sorted.index[i] in max_val:
      i += 1
    true += sorted.values[i]
    max_val.append(sorted.index[i])

  return (true/len(y))*100


In [16]:
acc_clustering(cm,tar)

76.12687813021702

In [17]:
y_pred

array([7, 8, 5, ..., 8, 6, 6])

In [18]:
tar

array([0, 1, 2, ..., 8, 9, 8])